# $N + \sqrt{N}$ Upper Limits
 >__Created__:  29 Sep. 2017 Harrison B. Prosper
 
 >__Updated__:  29 Jun. 2019, adaped for ENHEP 19, HBP


In this exercise, we shall determine the relative
frequency with which statements of the form  $$N + \sqrt{N} > \theta,$$
are true in an ensemble of 50,000 experiments, each associated with a _different_ mean count $\theta$.
We assume that each experiment yields a _single_ count $N$. Note that in the real world, unless the phenomenon being
investigated does not exist - in which case the mean count is zero, it is highly unlikely that every
experiment in a
random collection of experiments would be associated with _exactly_ the same mean count. 

We shall simulate 
 such an ensemble of experiments by 
sampling their _mean_ counts from a uniform distribution,
$\textrm{uniform}(0, b) = 1 \, / \, b$,
with mean $b = 10$.  

__TRandom3__ will be used to generate the sequences of random numbers.
   * $N_\textrm{exp}$ number of experiments
   * $b$ range of uniform density
   
Each experiment obtains a count $N$. The statement $$N + \sqrt{N} > \theta,$$ is either _True_ of _False_, where $\theta$ is the mean count for that experiment. In the real world, we typically do not know the mean count $\theta$ associated with an experiment. However, in a simulated world we usually do. Therefore, we can determine whether or not each statement is true. In the limit of an infinitely large ensemble of experiments, the relative frequency with which statements of the form $N + \sqrt{N} > \theta$ are true is called the __coverage__ probability. 

Note that the coverage is a property of the *ensemble* to which the statements belong and *not* a property of any given statement. Consequently, if a given statement is regarded as being embedded in a different ensemble, then, in general, the coverage probability will change. This is an example of the *reference class problem*, the fact that probabilities depend on the ensemble to which a finite set of outcomes are presumed to belong. The point is that absolute probabilities do not exist; they are all conditional.

__The Frequentist Principle__ The goal of frequentist analyses is to guarantee the following: over an (infinite) ensemble of statements, *which could be about different things*, a minimum fraction, CL, of these statements are true. The CL is called the __confidence level__. In 1937, Jerzy Neyman devised a procedure in which the CL is specified _a priori_. Consider statements of the form $x + \sigma > \mu$, where $\mu$ is the mean of a Gaussian--which can vary from one  experiment to the next, and $\sigma$ is the associated standard deviation. The coverage probability, as well as the confidence level, of such statements is 0.842.


In [2]:
import os, sys
import ROOT as rt
%jsroot off

In [33]:
Nexp = 50000  # number of experiments/statements
b    = 3.0    # range of uniform distribution
ran  = rt.TRandom3() # This has a cycle of 2^19937 - 1 ~ 10^6001

__Model the experiments__

In the following, we use a Python programming construct called __list comprehension__ to create one Python list from another. The syntax is
```python
    alist=[expression for loop expression involving a list]
```


In [34]:
def performExperiments(Nexp, b, ran):
    from math import sqrt, exp
    
    # generate Nexp experiments,each with a different mean value
    theta = [ ran.Uniform(0, b) for i in range(Nexp) ]
    
    # generate Nexp experimental outcomes
    N  = [ ran.Poisson(mean) for mean in theta ]

    # for each experiment compute an upper limit
    U = [ x + sqrt(x) for x in N ] 

    return (theta, N, U)        

In [35]:
theta, N, U = performExperiments(Nexp, b, ran)

K   = 5
fmt = ' %5.2f' * K
print 'theta', fmt % tuple(theta[:K])
print 'N    ', fmt % tuple(N[:K])
print 'U    ', fmt % tuple(U[:K])

theta   3.00  0.49  0.85  2.84  0.69
N       2.00  1.00  0.00  2.00  1.00
U       3.41  2.00  0.00  3.41  2.00


__Analyze results of experiments__ 

Relative frequency $p = k \, / \, n$ with rough measure of uncertainty $\sqrt{n p (1 - p)} \, / \, n$.

In [36]:
def computeCoverage(theta, U): 
    from math import sqrt
    
    # number of experiments
    n = len(theta)
    
    # count number of true statements in ensemble
    t  = [ u > t for (u, t) in zip(U, theta) ]
    
    # compute coverage (i.e., fraction of true statements)
    k  = sum(t)
    p  = float(k)/n
    
    # since we have k true statements our of n, this is a binomial
    # problem with variance n*p*(1-p). Therefore, a rough estimate
    # of the uncertainty in p is
    dp = sqrt(n*p*(1-p))/n
    
    return (p, dp)

In [37]:
q = (1+0.683)/2
results = computeCoverage(theta, U)
print "coverage: %8.3f %8.3f" % results
print "expected: %8.3f" % q

coverage:    0.612    0.002
expected:    0.842
